**Import Libiraries**

In [ ]:
import cv2 as cv
import numpy as np
from ultralytics import YOLO
import pyttsx3    #text to speach
import threading  #for multi threading 
import queue
import cvzone
import speech_recognition as sr
 

**Download model for pose estimation**

In [ ]:
model =YOLO(r"C:\yolo11n-pose.pt")
cap=cv.VideoCapture(0)


In [ ]:
up_thresh = 170
down_thresh = 45
triceps_up_thresh = 160
triceps_down_thresh = 90
leg_up_thresh = 170
leg_down_thresh = 90

push_up_left = False
push_up_right = False
combine = False
triceps_down = False
leg_down = False

left_hand_counter = 0
right_hand_counter = 0
combine_counter = 0
triceps_counter = 0
leg_counter = 0

mode = None

In [ ]:
engine= pyttsx3.init()
voices=engine.getProperty('voices')
engine.setProperty('voice',voices[0].id)
engine.setProperty('rate',150)
speech_queue=queue.Queue()


**Calculating angles**


In [ ]:
def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    
    radians= np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    #we have to convert the angle to degrees
    angle=np.abs(radians*180.0/np.pi)
    
    if angle>180.0:   #degrees
        angle=360-angle #to decide the wider angle
    return angle    

**Adapting speech to text and reverse**

In [ ]:
#speaking function
def speak(text):
    speech_queue.put(text)

In [ ]:
def worker_speak():
    while True:
        text= speech_queue.get()
        if text is None:
            break
        engine.say(text)
        engine.runAndWait()

**Training modes**

In [ ]:
def set_mode(new_mode):
    global mode
    mode = new_mode

def listen_commands():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()
    while True:
        with mic as source:
            recognizer.adjust_for_ambient_noise(source)  # to filter noices
                print("im listening")
                audio = recognizer.listen(source)
                commands = recognizer.recognize_google(audio).lower()
                print(commands)
                if "normal" in commands:
                    speak("normal mode started")
                    set_mode("normal")
                elif "combine" in commands:
                    speak("combine mode started")
                    set_mode("combine")
                elif "triceps" in commands:
                    speak("triceps mode started")
                    set_mode("triceps")
                elif "legs" in commands:
                    speak("legs mode started")
                    set_mode("legs")
                elif "stop" in commands:
                    speak("take care and goodbye")
                    set_mode("stop")
                    break 
   
            

**Threading processes**

In [ ]:
thread_speak=threading.Thread(target=worker_speak,daemon=True)
thread_speak.start()
thread_listen=threading.Thread(target=listen_commands,daemon=True)
thread_listen.start()

**Main loop for video capturing**

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv.resize(frame, (720, 480))
    result = model.track(frame)

    if result[0].keypoints is not None:
        Keypoints = result[0].keypoints.xy.cpu().numpy()  # to convert the tensor to numpy array [[[]]]

        for keypoint in Keypoints:
            if len(keypoint) > 0:
                for i, point in enumerate(keypoint):
                    cx, cy = int(point[0]), int(point[1])
                    cv.circle(frame, (cx, cy), 5, (255, 0, 0), -1)
                    cvzone.putTextRect(frame, f'{i}', (cx, cy), 1, 1)

                if mode and len(keypoint) > 15:  # Ensure enough keypoints are detected for legs
                    # Arm keypoints
                    left_shoulder = (int(keypoint[5][0]), int(keypoint[5][1]))
                    left_elbow = (int(keypoint[7][0]), int(keypoint[7][1]))
                    left_wrist = (int(keypoint[9][0]), int(keypoint[9][1]))
                    right_shoulder = (int(keypoint[6][0]), int(keypoint[6][1]))
                    right_elbow = (int(keypoint[8][0]), int(keypoint[8][1]))
                    right_wrist = (int(keypoint[10][0]), int(keypoint[10][1]))

                    # Leg keypoints
                    left_hip = (int(keypoint[11][0]), int(keypoint[11][1]))
                    left_knee = (int(keypoint[13][0]), int(keypoint[13][1]))
                    left_ankle = (int(keypoint[15][0]), int(keypoint[15][1]))

                    if mode == "triceps":
                        right_hand_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                        cvzone.putTextRect(frame, f'Right Arm Angle: {int(right_hand_angle)}', (50, 80), 1, 1, colorR=(0, 0, 255))

                        if right_hand_angle > triceps_up_thresh and not triceps_down:
                            triceps_down = True
                        elif right_hand_angle < triceps_down_thresh and triceps_down:
                            triceps_counter += 1
                            speak(f'triceps {triceps_counter}')
                            triceps_down = False

                    elif mode == "legs":
                        leg_angle = calculate_angle(left_hip, left_knee, left_ankle)
                        cvzone.putTextRect(frame, f'Leg Angle: {int(leg_angle)}', (50, 50), 1, 1, colorR=(255, 0, 0))

                        if leg_angle < leg_down_thresh and not leg_down:
                            leg_down = True
                        elif leg_angle > leg_up_thresh and leg_down:
                            leg_counter += 1
                            speak(f'leg {leg_counter}')
                            leg_down = False

                    elif mode in ["normal", "combine"]:
                        left_hand_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                        right_hand_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

                        cvzone.putTextRect(frame, f'Left Arm Angle: {int(left_hand_angle)}', (50, 50), 1, 1, colorR=(255, 0, 0))
                        cvzone.putTextRect(frame, f'Right Arm Angle: {int(right_hand_angle)}', (50, 80), 1, 1, colorR=(0, 0, 255))

                        if mode == "normal":
                            if left_hand_angle < down_thresh and not push_up_left:
                                push_up_left = True
                            elif left_hand_angle > up_thresh and push_up_left:
                                left_hand_counter += 1
                                speak(f'left {left_hand_counter}')
                                push_up_left = False

                            if right_hand_angle < down_thresh and not push_up_right:
                                push_up_right = True
                            elif right_hand_angle > up_thresh and push_up_right:
                                right_hand_counter += 1
                                speak(f'Right {right_hand_counter}')
                                push_up_right = False

                        elif mode == "combine":
                            if right_hand_angle <= down_thresh and left_hand_angle <= down_thresh and not combine:
                                combine = True
                            elif left_hand_angle >= up_thresh and right_hand_angle >= up_thresh and combine:
                                combine_counter += 1
                                speak(f"combine {combine_counter}")
                                combine = False

    if mode == 'normal':
        cvzone.putTextRect(frame, f'Left hand counter: {int(left_hand_counter)}', (50, 110), 1, 1, colorR=(0, 0, 0))
        cvzone.putTextRect(frame, f'Right hand counter: {int(right_hand_counter)}', (50, 140), 1, 1, colorR=(0, 0, 0))
    elif mode == 'combine':
        cvzone.putTextRect(frame, f'Combine counter: {int(combine_counter)}', (50, 110), 1, 1, colorR=(0, 0, 0))
    elif mode == 'triceps':
        cvzone.putTextRect(frame, f'Triceps counter: {int(triceps_counter)}', (50, 110), 1, 1, colorR=(0, 0, 0))
    elif mode == 'legs':
        cvzone.putTextRect(frame, f'Leg counter: {int(leg_counter)}', (50, 110), 1, 1, colorR=(0, 0, 0))

    cv.imshow("RGB", frame)

    key = cv.waitKey(1)
    if mode == "stop":
        break
cap.release()
cv.destroyAllWindows()